# Naive Bayes

# Importing the libraries

In [80]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
from skopt import BayesSearchCV
import matplotlib.pyplot as plt
import numpy as np
import joblib

# Load the dataset and set display options to show all rows and print first few rows


In [81]:
# Load the dataset
os.chdir(r'/home/zamlamb/KdG/Data n AI 5/Team-Stress/Resources')
df = pd.read_csv('processed_features_job_satisfaction.csv')

# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Print the first few rows of the DataFrame
print(df.head())

  JobSatisfaction  Workload_Binned  SleepHours_Binned Age_Binned  \
0  Very Satisfied                2                  7      30-39   
1  Very Satisfied                2                  7      30-39   
2  Very Satisfied                5                  6      20-29   
3  Very Satisfied                3                  7      20-29   
4  Very Satisfied                2                  4      20-29   

   Stress_Binned Experience_Binned        JobLevel  Gender MaritalStatus  \
0              1              6-10             Mid    Male       Married   
1              2             11-15             Mid  Female       Married   
2              4               1-5  Intern/Fresher  Female        Single   
3              1              6-10          Junior  Female       Married   
4              1               1-5          Junior   Other        Single   

        Dept    EmpType haveOT_Binned  
0         IT  Full-Time           Yes  
1    Finance  Full-Time            No  
2  Marketing  

# List the significant features, and separate features and the target variable

In [82]:
# List of significant features
significant_features = ['Workload_Binned', 'SleepHours_Binned', 'Stress_Binned']

# Separate features and target variable
X = df[significant_features]
le = LabelEncoder()
y = le.fit_transform(df['JobSatisfaction']).ravel()  # Ensure y is a 1D array


# Define the hyperparameter search space for Naive Bayes


In [83]:
search_space_gnb = {
    'var_smoothing': (1e-15, 1e-4, 'log-uniform')
}

# Initialize and perform Bayesian optimization for Gaussian Naive Bayes


In [84]:
gnb = GaussianNB()
opt_gnb = BayesSearchCV(gnb, search_space_gnb, n_iter=16, cv=StratifiedKFold(n_splits=5), n_jobs=-1)
opt_gnb.fit(X, y)

/home/zamlamb/KdG/Data n AI 5/Team-Stress/venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [1.168874487046123e-15] before, using random point [1.4424961226206872e-07]
  warnings.warn(
/home/zamlamb/KdG/Data n AI 5/Team-Stress/venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [4.704284823422881e-09] before, using random point [5.299189465490654e-07]
  warnings.warn(
/home/zamlamb/KdG/Data n AI 5/Team-Stress/venv/lib/python3.12/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [7.62706148354898e-12] before, using random point [3.936686909566968e-15]
  warnings.warn(


BayesSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
              estimator=GaussianNB(), n_iter=16, n_jobs=-1,
              search_spaces={'var_smoothing': (1e-15, 0.0001, 'log-uniform')})

# Print the best hyperparameters for Gaussian Naive Bayes

In [85]:
print(f"Best hyperparameters for Gaussian Naive Bayes: {opt_gnb.best_params_}")

Best hyperparameters for Gaussian Naive Bayes: OrderedDict({'var_smoothing': 1.0844600358615302e-09})


# Evaluate the Gaussian Naive Bayes model


In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
opt_gnb.best_estimator_.fit(X_train, y_train)
y_test_pred = opt_gnb.best_estimator_.predict(X_test)
y_test_prob = opt_gnb.best_estimator_.predict_proba(X_test)


# Calculate AUC and accuracy


In [87]:
final_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr')
final_acc = accuracy_score(y_test, y_test_pred)

print(f"Final Evaluation on Test Set for Gaussian Naive Bayes Model:\nAUC: {final_auc:.4f}, Accuracy: {final_acc:.4f}")

Final Evaluation on Test Set for Gaussian Naive Bayes Model:
AUC: 0.6367, Accuracy: 0.4252


# Save results to a CSV file


In [88]:
results_df = pd.DataFrame({
    'True Label': y_test,
    'Predicted Label': y_test_pred,
    'Probability': y_test_prob.max(axis=1)
})
results_df.to_csv('gnb_model_results.csv', index=False)


# Save the best Gaussian Naive Bayes model to a joblib file

In [89]:
joblib.dump(opt_gnb.best_estimator_, 'gnb_model.pkl')


['gnb_model.pkl']

# Plot ROC curve for the Gaussian Naive Bayes model


# Conclusion is that even if I try to optimize the hyperparameters, it still provides the same accuracy